In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created and saved")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sibasish.padhy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sibasish.padhy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sibasish.padhy\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


81 documents
18 classes ['Active Rules', 'Applications', 'Cluster', 'Current', 'Demand', 'Granularity', 'Plugin', 'Python', 'Styles', 'The Granularity of the Measure group', 'creator', 'greeting', 'invalid', 'joke', 'noanswer', 'options', 'python_is_best', 'thanks']
131 unique lemmatized words ["'s", ',', '.', 'a', 'active', 'add', 'am', 'an', 'and', 'anyone', 'aot', 'application', 'applied', 'are', 'at', 'attribute', 'awesome', 'basic', 'be', 'beginner', 'best', 'bored', 'can', 'chat', 'cluster', 'code', 'could', 'created', 'creator', 'current', 'currently', 'daddy', 'dance/', 'date', 'day', 'define', 'demand', 'demandforecasmeasuregroup', 'demandforecastmeasuregroup', 'different', 'dimension', 'do', 'ever', 'explain', 'father', 'for', 'fun', 'funny', 'generate', 'give', 'good', 'granularity', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'info', 'information', 'is', 'is/are', 'joke', 'know', 'l', 'language', 'laugh', 'layer', 'learn', 'list', 'love', '

C:\Users\SIBASI~1.PAD\AppData\Local\Temp/ipykernel_57744/3069456881.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
C:\Users\sibasish.padhy\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


17/17 [==============================] - 0s 1ms/step - loss: 2.9046 - accuracy: 0.0741
Epoch 2/200
17/17 [==============================] - 0s 1ms/step - loss: 2.8282 - accuracy: 0.0988
Epoch 3/200
17/17 [==============================] - 0s 1ms/step - loss: 2.7715 - accuracy: 0.1235
Epoch 4/200
17/17 [==============================] - 0s 1ms/step - loss: 2.6654 - accuracy: 0.2099
Epoch 5/200
17/17 [==============================] - 0s 946us/step - loss: 2.5499 - accuracy: 0.2222
Epoch 6/200
17/17 [==============================] - 0s 1ms/step - loss: 2.4566 - accuracy: 0.2840
Epoch 7/200
17/17 [==============================] - 0s 1ms/step - loss: 2.2585 - accuracy: 0.3951
Epoch 8/200
17/17 [==============================] - 0s 1ms/step - loss: 2.1277 - accuracy: 0.3580
Epoch 9/200
17/17 [==============================] - 0s 1ms/step - loss: 2.0322 - accuracy: 0.4198
Epoch 10/200
17/17 [==============================] - 0s 1ms/step - loss: 1.7639 - accuracy: 0.5185
Epoch 11/200
17/17 

17/17 [==============================] - 0s 936us/step - loss: 0.1098 - accuracy: 0.9630
Epoch 83/200
17/17 [==============================] - 0s 940us/step - loss: 0.0670 - accuracy: 1.0000
Epoch 84/200
17/17 [==============================] - 0s 935us/step - loss: 0.1363 - accuracy: 0.9630
Epoch 85/200
17/17 [==============================] - 0s 935us/step - loss: 0.0899 - accuracy: 0.9753
Epoch 86/200
17/17 [==============================] - 0s 971us/step - loss: 0.1034 - accuracy: 0.9753
Epoch 87/200
17/17 [==============================] - 0s 935us/step - loss: 0.1111 - accuracy: 0.9753
Epoch 88/200
17/17 [==============================] - 0s 948us/step - loss: 0.1106 - accuracy: 0.9630
Epoch 89/200
17/17 [==============================] - 0s 959us/step - loss: 0.0914 - accuracy: 0.9877
Epoch 90/200
17/17 [==============================] - 0s 935us/step - loss: 0.1498 - accuracy: 0.9753
Epoch 91/200
17/17 [==============================] - 0s 911us/step - loss: 0.0688 - accuracy: 

17/17 [==============================] - 0s 1ms/step - loss: 0.1035 - accuracy: 0.9506
Epoch 163/200
17/17 [==============================] - 0s 874us/step - loss: 0.0314 - accuracy: 1.0000
Epoch 164/200
17/17 [==============================] - 0s 935us/step - loss: 0.0267 - accuracy: 0.9877
Epoch 165/200
17/17 [==============================] - 0s 890us/step - loss: 0.0802 - accuracy: 0.9753
Epoch 166/200
17/17 [==============================] - 0s 893us/step - loss: 0.0684 - accuracy: 0.9753
Epoch 167/200
17/17 [==============================] - 0s 875us/step - loss: 0.0326 - accuracy: 1.0000
Epoch 168/200
17/17 [==============================] - 0s 936us/step - loss: 0.0553 - accuracy: 1.0000
Epoch 169/200
17/17 [==============================] - 0s 920us/step - loss: 0.0267 - accuracy: 1.0000
Epoch 170/200
17/17 [==============================] - 0s 908us/step - loss: 0.0313 - accuracy: 1.0000
Epoch 171/200
17/17 [==============================] - 0s 1ms/step - loss: 0.0366 - accur

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text=">", width="12", height=4,
                    bd=0, bg="#25cdf7", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()
